# Exploring QSTS Results Capabilities with `py_dss_toolkit`: A Hands-on Example
This notebook demonstrates the process of solving a QSTS (Quasi-Static Time Series) simulation using the `py_dss_toolkit` package, and then retrieve results from enerymeters and monitors.

In [1]:
import py_dss_interface
from py_dss_toolkit import dss_tools

Define the path to the DSS file

In [2]:
import ipynbname
nb_path = ipynbname.path()
dss_file = nb_path.parent.parent.joinpath("feeders", "123bus", "IEEE123Master.dss")

Create an instance of DSS

In [3]:
dss = py_dss_interface.DSS()

Connect DSS instance to the dss_tools

In [4]:
dss_tools.update_dss(dss)

Compile the DSS model from the specified file, load bus coordinates for visualization

In [5]:
dss.text(f"compile [{dss_file}]")
dss.text(f"buscoords buscoords.dat")

''

Add a line and monitors at the feeder head. A short line (with almost zero impedance) is added between the `Vsource.source` and the first element(s) to ensure only one PD element is connected to the `Vsource.source`. This allows for the addition of an energy meter and monitors for voltage and power at the feeder head to capture results for the whole system.

In [6]:
dss_tools.model.add_line_in_vsource(add_meter=True, add_monitors=True)

We will define the default loadshape to all loads using the `model.batchedit`.

In [7]:
dss_tools.model.batchedit("load", "daily", "default")

We will solve the QSTS simulation for the system using a method from `dss_tools`. Parameters in the `solve_daily` method can be modified if necessary.

In [8]:
dss_tools.simulation.solve_daily()

We will retrieve results for all meters in the system.

In [9]:
energymeters_df = dss_tools.results.energymeters
energymeters_df

name           kwh         kvarh       max kw     max kva  \
0  meter_feeder_head  71364.680499  22087.766777  3613.557978  3845.26618   

      zone kwh    zone kvarh  zone max kw  zone max kva  overload kwh normal  \
0  69731.87289  38374.634903  3516.771626   4014.043033         18212.246572   

   ...  aux19  aux24  aux29  4.16 kv load energy  aux5  aux10  aux15  aux20  \
0  ...    0.0    0.0    0.0          69731.87289   0.0    0.0    0.0    0.0   

   aux25  aux30  
0    0.0    0.0  

[1 rows x 68 columns]

We will retrieve results from the monitors at the feeder head.

In [10]:
monitor_pq_df = dss_tools.results.monitor("monitor_feeder_head_pq")
monitor_pq_df

Hour  sec      P1 (kW)   Q1 (kvar)     P2 (kW)   Q2 (kvar)      P3 (kW)  \
0    1.0  0.0   978.783691  282.561920  645.843079  132.141373   796.224854   
1    2.0  0.0   904.867798  236.119217  597.277954   99.776741   736.018250   
2    3.0  0.0   880.225159  220.761841  581.303040   89.175758   716.239868   
3    4.0  0.0   842.684509  200.826111  557.355835   73.282158   685.460266   
4    5.0  0.0   838.375916  198.155350  554.493042   71.390923   681.949341   
5    6.0  0.0   870.205872  217.917847  575.506104   85.294144   707.927490   
6    7.0  0.0   948.284302  266.748505  627.021545  119.538628   771.675964   
7    8.0  0.0  1084.483398  345.865234  715.586792  175.344437   881.588562   
8    9.0  0.0  1205.712891  424.023834  795.441711  229.443604   980.673340   
9   10.0  0.0  1281.870483  466.223267  844.344177  259.122650  1042.966187   
10  11.0  0.0  1368.496094  523.340088  901.327271  298.264099  1113.901978   
11  12.0  0.0  1445.478027  570.229797  951.823669  329.338989  1176.769775   
12  13.0  0.0  1439.712891  566.377258  948.017395  326.701416  1172.079956   
13  14.0  0.0  1432.475220  561.544312  943.255676  323.404022  1166.149780   
14  15.0  0.0  1446.508179  570.918335  952.580078  329.862213  1177.690918   
15  16.0  0.0  1459.850708  579.846008  961.342590  335.938538  1188.618408   
16  17.0  0.0  1461.398560  580.883850  962.299927  336.603394  1189.859497   
17  18.0  0.0  1400.188599  540.094543  921.388367  311.674103  1140.119751   
18  19.0  0.0  1367.712158  518.601929  900.485962  297.268127  1113.931274   
19  20.0  0.0  1334.490967  496.674957  878.510986  282.193085  1085.138062   
20  21.0  0.0  1279.043579  463.661316  843.441284  258.127441  1041.247681   
21  22.0  0.0  1279.101196  463.699493  843.412964  258.110291  1041.253296   
22  23.0  0.0  1207.605347  420.593292  797.022339  229.862061   984.405884   
23  24.0  0.0  1096.814087  356.701263  725.351868  184.983429   893.986877   

     Q3 (kvar)  
0   168.142700  
1   132.165070  
2   120.398048  
3   105.504631  
4   103.425514  
5   118.825333  
6   156.797073  
7   219.150909  
8   279.316681  
9   310.278961  
10  353.989319  
11  389.007172  
12  386.091492  
13  382.407684  
14  389.582489  
15  396.379425  
16  397.149872  
17  366.283264  
18  350.091644  
19  335.686768  
20  308.756470  
21  308.759033  
22  274.045746  
23  226.320282

In [11]:
monitor_vi_df = dss_tools.results.monitor("monitor_feeder_head_vi")
monitor_vi_df

Hour  sec           V1   VAngle1           V2     VAngle2           V3  \
0    1.0  0.0  2401.765381 -0.000972  2401.771729 -120.000641  2401.770020   
1    2.0  0.0  2401.767334 -0.000899  2401.772949 -120.000595  2401.771729   
2    3.0  0.0  2401.767822 -0.000874  2401.773438 -120.000580  2401.772217   
3    4.0  0.0  2401.768799 -0.000837  2401.774170 -120.000557  2401.772705   
4    5.0  0.0  2401.768799 -0.000833  2401.774170 -120.000549  2401.772705   
5    6.0  0.0  2401.768066 -0.000864  2401.773682 -120.000572  2401.772217   
6    7.0  0.0  2401.766113 -0.000942  2401.772217 -120.000626  2401.770508   
7    8.0  0.0  2401.762695 -0.001077  2401.769775 -120.000710  2401.768066   
8    9.0  0.0  2401.759521 -0.001198  2401.767578 -120.000793  2401.765381   
9   10.0  0.0  2401.757812 -0.001273  2401.766357 -120.000839  2401.764160   
10  11.0  0.0  2401.755371 -0.001359  2401.764648 -120.000893  2401.762451   
11  12.0  0.0  2401.753418 -0.001436  2401.763428 -120.000946  2401.760986   
12  13.0  0.0  2401.753418 -0.001430  2401.763428 -120.000938  2401.760986   
13  14.0  0.0  2401.753662 -0.001423  2401.763672 -120.000938  2401.761230   
14  15.0  0.0  2401.753418 -0.001437  2401.763428 -120.000946  2401.760986   
15  16.0  0.0  2401.752930 -0.001450  2401.763184 -120.000954  2401.760498   
16  17.0  0.0  2401.752930 -0.001452  2401.763184 -120.000954  2401.760498   
17  18.0  0.0  2401.754639 -0.001391  2401.764160 -120.000916  2401.761963   
18  19.0  0.0  2401.755615 -0.001358  2401.764648 -120.000893  2401.762451   
19  20.0  0.0  2401.756348 -0.001325  2401.765381 -120.000870  2401.763184   
20  21.0  0.0  2401.757812 -0.001270  2401.766357 -120.000839  2401.764160   
21  22.0  0.0  2401.757812 -0.001270  2401.766357 -120.000839  2401.764160   
22  23.0  0.0  2401.759521 -0.001199  2401.767578 -120.000793  2401.765625   
23  24.0  0.0  2401.762207 -0.001089  2401.769531 -120.000717  2401.767578   

       VAngle3          I1    IAngle1          I2     IAngle2          I3  \
0   119.999207  424.168640 -16.103703  274.473511 -131.563934  338.827179   
1   119.999268  389.366364 -14.625730  252.128143 -129.484421  311.349518   
2   119.999290  377.841156 -14.080334  244.862122 -128.722137  302.397003   
3   119.999321  360.685944 -13.405363  234.057343 -127.490944  288.758484   
4   119.999321  358.683685 -13.299000  232.773727 -127.337013  287.182709   
5   119.999298  373.506775 -14.059810  242.234467 -128.430847  298.875397   
6   119.999237  410.151428 -15.712018  265.768158 -130.794266  327.860046   
7   119.999123  473.943604 -17.689686  306.755676 -133.768951  378.229370   
8   119.999023  532.151550 -19.376898  344.692780 -136.090897  424.552460   
9   119.998962  567.926575 -19.987875  367.733948 -137.061676  453.059174   
10  119.998894  610.033264 -20.929205  395.290924 -138.311127  486.641327   
11  119.998833  646.980774 -21.530283  419.354553 -139.086990  516.038269   
12  119.998833  644.159485 -21.475906  417.498077 -139.015686  513.803467   
13  119.998840  640.618896 -21.407089  415.176849 -138.925690  510.978912   
14  119.998833  647.484985 -21.539953  419.723419 -139.101044  516.477600   
15  119.998817  654.018738 -21.664200  424.000519 -139.262787  521.687561   
16  119.998817  654.777161 -21.678524  424.468140 -139.280304  522.279297   
17  119.998871  624.852844 -21.094505  404.983673 -138.689835  498.597595   
18  119.998894  609.026123 -20.766823  394.828186 -138.269974  486.163879   
19  119.998924  592.866699 -20.415739  384.184570 -137.808807  472.933472   
20  119.998962  566.456116 -19.927168  367.253082 -137.017090  452.192780   
21  119.998962  566.484070 -19.927855  367.239716 -137.016556  452.195312   
22  119.999023  532.423401 -19.203764  345.373383 -136.088409  425.453461   
23  119.999115  480.213623 -18.016434  311.673584 -134.307663  383.962799   

       IAngle3  
0   108.074974  
1   109.819290  
2   110.457230  
3   111.249138  
4   111.375473  
5   110.471046  
6   108.

### Conclusion
In this notebook, we explored the QSTS simulation Results capabilities of py_dss_toolkit. 
We compiled and solved a DSS model, then retrieved results for energymeter and monitors.
If you have suggestions or ideas on what could be added to this tool, please don't hesitate to reach out. Your feedback is invaluable in helping improve this tool's functionality.